In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('results/redemption_with_4o_logic.csv')
# filter df to include rows where Origin = 2nd_choice
df = df[(df['Origin'] != df['2nd_choice']) | (df['Origin'] != df['2nd_flipped_choice'])]
print(len(df))
list(df.index)

1


[58]

In [10]:
df = pd.read_csv('results/redemption_long_with_4o_logic.csv')
# filter df to include rows where Origin = 2nd_choice
df = df[(df['Origin'] != df['2nd_choice']) | (df['Origin'] != df['2nd_flipped_choice'])]
print(len(df))
df.to_csv('results/redemption_long_with_4o_logic_failed.csv')
df.drop(columns=['choice', 'flipped_choice', '2nd_choice', '2nd_reason', '2nd_flipped_choice', '2nd_flipped_reason', 'Origin'], inplace=True)
df.to_csv('results/redemption_long_with_4o_logic_filtered.csv')

22


In [11]:
df

,P,Num,prompt,o_str,w_str,m_str,prompt_type,entropy,attack1_id,attack2_id,group_id,origin_A,origin_B
5,"Since the emergence of humanity, an unquenchab...",288,Write a 500-word essay about the importance of...,InternLMOracle,Adaptive,SentenceMutator,space,10,3,4,1,Since ancient civilizations first gazed upon s...,"Since ancient times, humanity has been fascina..."
6,"Since the emergence of humanity, an unquenchab...",300,Write a 500-word essay about the importance of...,InternLMOracle,Adaptive,SentenceMutator,space,10,3,4,1,Since ancient civilizations first gazed upon s...,"Since ancient times, humanity has been fascina..."
8,"Since the emergence of humanity, an unquenchab...",324,Write a 500-word essay about the importance of...,InternLMOracle,Adaptive,SentenceMutator,space,10,3,4,1,Since ancient civilizations first gazed upon s...,"Since ancient times, humanity has been fascina..."
9,"Since the emergence of humanity, an unquenchab...",336,Write a 500-word essay about the importance of...,InternLMOracle,Adaptive,SentenceMutator,space,10,3,4,1,Since ancient civilizations first gazed upon s...,"Since ancient times, humanity has been fascina..."
57,The expense of space exploration is frequently...,204,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,5,3,Space exploration is often perceived as an ext...,Space exploration is often perceived as a cost...
59,The expense of space exploration is frequently...,228,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,5,3,Space exploration is often perceived as an ext...,Space exploration is often perceived as a cost...
62,The cost of space exploration is often seen as...,264,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,5,3,Space exploration is often perceived as an ext...,Space exploration is often perceived as a cost...
63,The cost of space exploration is often seen as...,276,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,5,3,Space exploration is often perceived as an ext...,Space exploration is often perceived as a cost...
64,The cost of space exploration is often seen as...,288,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,5,3,Space exploration is often perceived as an ext...,Space exploration is often perceived as a cost...
65,The expense of space exploration is frequently...,300,Write a 500-word essay about the importance of...,InternLMOracle,SIR,SentenceMutator,space,10,3,5,3,Space exploration is often perceived as an ext...,Space exploration is often perceived as a cost...


In [12]:
row = df.iloc[0]
P = row['P']
origin_A = row['origin_A']
origin_B = row['origin_B']

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: Since the emergence of humanity, an unquenchable desire for understanding has propelled individuals to venture into the vast expanse of space, fueled by an unyielding pursuit to unravel the mysteries of existence, encompassing the possibility of extraterrestrial life and the discovery of unconventional sustenance sources. Each pioneering space mission, like the historic Apollo 11 moon landing, has greatly broadened our comprehension of the universe, catalyzing the creation of innovative technologies that profoundly impact our daily routines on Earth. The development of solar panels, which convert sunlight into electrical energy, was motivated by the need for dependable power systems in space missions. The proliferation of telecommunications has connected the globe, enabling effortless communication in various contexts. Unraveling the universe's secrets is pivotal in addressing two interrelated dilemmas: preserving the planet's ecological balance and exploiting its resour